In [1]:
import pandas as pd
import numpy as np

In [2]:
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

Pandas: 2.3.3
NumPy: 2.3.4


In [ ]:
import pandas as pd
import numpy as np

def analizar_dataframe(df):
    """
    Análisis eficiente de DataFrame usando operaciones vectorizadas.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame a analizar
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame con el análisis de cada columna
    """
    
    # Pre-calcular información común para todas las columnas
    total_filas = len(df)
    
    # Lista para almacenar resultados
    resultados = []
    
    for columna in df.columns:
        col_data = df[columna]
        
        # Tipo de dato (más eficiente con select_dtypes)
        es_numerica = pd.api.types.is_numeric_dtype(col_data)
        tipo = 'Numérica' if es_numerica else 'Alfanumérica'
        
        # Máscara de nulos (calcular una sola vez)
        mask_nulos = col_data.isna()
        cantidad_nulos = mask_nulos.sum()
        
        # Valores distintos (incluidos null) - muy eficiente
        valores_distintos = col_data.nunique(dropna=False)
        
        # Inicializar contadores
        cantidad_blancos = 0
        cantidad_ceros = 0
        cantidad_negativos = 0
        
        if es_numerica:
            # Operaciones vectorizadas para numéricos (muy rápido)
            col_no_nulos = col_data.dropna()
            if len(col_no_nulos) > 0:
                cantidad_ceros = (col_no_nulos == 0).sum()
                cantidad_negativos = (col_no_nulos < 0).sum()
        else:
            # Para alfanuméricos, usar operaciones de string vectorizadas
            # Más eficiente que apply
            if col_data.dtype == 'object':
                # Filtrar solo strings y contar blancos en una operación
                mask_strings = col_data.apply(lambda x: isinstance(x, str))
                if mask_strings.any():
                    cantidad_blancos = col_data[mask_strings].str.strip().eq('').sum()
        
        # 5 valores de ejemplo (optimizado)
        valores_unicos = col_data.dropna().unique()
        valores_ejemplo = valores_unicos[:5] if len(valores_unicos) > 0 else []
        valores_ejemplo_str = ', '.join(map(str, valores_ejemplo))
        
        # Agregar resultados
        resultados.append({
            'Columna': columna,
            'Tipo': tipo,
            'Total Valores': total_filas,
            'Nulos': int(cantidad_nulos),
            'Valores Distintos': int(valores_distintos),
            'Valores Blancos': int(cantidad_blancos),
            'Valores Cero': int(cantidad_ceros),
            'Valores Negativos': int(cantidad_negativos),
            'Ejemplos': valores_ejemplo_str
        })
    
    return pd.DataFrame(resultados)


def analizar_csv(ruta_archivo, separador=',', encoding='utf-8', nrows=None):
    """
    Lee y analiza un archivo CSV de forma eficiente.
    
    Parameters:
    -----------
    ruta_archivo : str
        Ruta del archivo CSV
    separador : str, default=','
        Separador del CSV
    encoding : str, default='utf-8'
        Codificación del archivo
    nrows : int, optional
        Número de filas a leer (útil para archivos grandes)
        
    Returns:
    --------
    tuple
        (DataFrame original, DataFrame de análisis)
    """
    
    print(f"Leyendo archivo: {ruta_archivo}")
    
    # Leer CSV con optimizaciones
    df = pd.read_csv(
        ruta_archivo, 
        sep=separador, 
        encoding=encoding,
        nrows=nrows,
        low_memory=False  # Más rápido para archivos grandes
    )
    
    print(f"✓ Archivo cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas\n")
    
    print("Analizando columnas...")
    df_analisis = analizar_dataframe(df)
    print("✓ Análisis completado\n")
    
    return df, df_analisis


# ============================================
# VERSIÓN ULTRA OPTIMIZADA (para Big Data)
# ============================================

def analizar_csv_chunked(ruta_archivo, chunksize=10000, separador=',', encoding='utf-8'):
    """
    Analiza archivos CSV muy grandes por chunks (evita cargar todo en memoria).
    Ideal para datasets de millones de filas.
    
    Parameters:
    -----------
    ruta_archivo : str
        Ruta del archivo CSV
    chunksize : int, default=10000
        Número de filas por chunk
    separador : str, default=','
        Separador del CSV
    encoding : str, default='utf-8'
        Codificación del archivo
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame con el análisis de cada columna
    """
    
    print(f"Analizando archivo por chunks de {chunksize:,} filas...")
    
    # Diccionarios para acumular estadísticas
    stats = {}
    total_rows = 0
    
    # Leer por chunks
    for i, chunk in enumerate(pd.read_csv(ruta_archivo, sep=separador, 
                                          encoding=encoding, chunksize=chunksize)):
        
        total_rows += len(chunk)
        
        for columna in chunk.columns:
            if columna not in stats:
                # Inicializar estadísticas para nueva columna
                es_numerica = pd.api.types.is_numeric_dtype(chunk[columna])
                stats[columna] = {
                    'tipo': 'Numérica' if es_numerica else 'Alfanumérica',
                    'nulos': 0,
                    'valores_unicos': set(),
                    'blancos': 0,
                    'ceros': 0,
                    'negativos': 0,
                    'ejemplos': []
                }
            
            col_data = chunk[columna]
            col_stats = stats[columna]
            
            # Acumular nulos
            col_stats['nulos'] += col_data.isna().sum()
            
            # Acumular valores únicos (limitado para eficiencia)
            if len(col_stats['valores_unicos']) < 10000:
                col_stats['valores_unicos'].update(col_data.dropna().unique())
            
            # Ejemplos (solo del primer chunk)
            if i == 0 and len(col_stats['ejemplos']) < 5:
                ejemplos = col_data.dropna().unique()[:5].tolist()
                col_stats['ejemplos'].extend(ejemplos)
            
            # Estadísticas específicas por tipo
            if col_stats['tipo'] == 'Numérica':
                col_no_nulos = col_data.dropna()
                col_stats['ceros'] += (col_no_nulos == 0).sum()
                col_stats['negativos'] += (col_no_nulos < 0).sum()
            else:
                if col_data.dtype == 'object':
                    mask_strings = col_data.apply(lambda x: isinstance(x, str))
                    if mask_strings.any():
                        col_stats['blancos'] += col_data[mask_strings].str.strip().eq('').sum()
        
        if (i + 1) % 10 == 0:
            print(f"  Procesados {total_rows:,} filas...")
    
    print(f"✓ Análisis completado: {total_rows:,} filas totales\n")
    
    # Construir DataFrame de resultados
    resultados = []
    for columna, col_stats in stats.items():
        resultados.append({
            'Columna': columna,
            'Tipo': col_stats['tipo'],
            'Total Valores': total_rows,
            'Nulos': int(col_stats['nulos']),
            'Valores Distintos': len(col_stats['valores_unicos']),
            'Valores Blancos': int(col_stats['blancos']),
            'Valores Cero': int(col_stats['ceros']),
            'Valores Negativos': int(col_stats['negativos']),
            'Ejemplos': ', '.join(map(str, col_stats['ejemplos'][:5]))
        })
    
    return pd.DataFrame(resultados)


# ============================================
# EJEMPLO DE USO
# ============================================

if __name__ == "__main__":
    
    # ===== PARA ARCHIVOS NORMALES (< 1 GB) =====
    ruta_csv = 'data/raw/tu_archivo.csv'
    df_original, df_analisis = analizar_csv(ruta_csv)
    
    print("=" * 100)
    print("ANÁLISIS DE COLUMNAS")
    print("=" * 100)
    print(df_analisis.to_string(index=False))
    
    # ===== PARA ARCHIVOS GRANDES (> 1 GB) =====
    # df_analisis = analizar_csv_chunked(ruta_csv, chunksize=50000)
    # print(df_analisis.to_string(index=False))
    
    # Guardar resultado
    df_analisis.to_csv('reports/analisis_columnas.csv', index=False)
    print("\n✓ Análisis guardado en: reports/analisis_columnas.csv")